In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

# TEST ZMIANY

# Zadanie 2

In [10]:
# Prawdopodobieństwa, że urządzenie nie ulegnie awarii w zależności od kwoty przeznaczonej na remont
probabilities = {
    'U1': [0.2, 0.2, 0.4, 0.6, 0.7],
    'U2': [0.1, 0.3, 0.5, 0.7, 0.8],
    'U3': [0.2, 0.4, 0.6, 0.8, 0.9],
    'U4': [0.1, 0.2, 0.3, 0.5, 0.7]
}

# Wartości dostępne do wydania na remont w tys. PLN
amounts = [0, 1, 2, 3, 4]  # Kwoty przeznaczone na remonty (0-4 tys. PLN)

# Budżet całkowity
total_budget = 10  # 10 tysięcy PLN

# Funkcja do obliczenia maksymalnego prawdopodobieństwa braku awarii oraz przydziału środków
def max_reliability_with_allocation(total_budget, probabilities):
    num_devices = len(probabilities)  # Liczba urządzeń
    
    # Tablica DP: dp[i][j] oznacza maksymalne prawdopodobieństwo dla i urządzeń i j tysięcy PLN
    dp = np.zeros((num_devices + 1, total_budget + 1))
    
    # Tablica przechowująca optymalny przydział środków
    allocation = np.zeros((num_devices + 1, total_budget + 1), dtype=int)
    
    # Inicjalizacja - dla 0 urządzeń prawdopodobieństwo awarii to 1 (nie ma urządzeń do naprawy)
    dp[0, 0] = 1  # Nie ma urządzeń, więc prawdopodobieństwo braku awarii to 1

    # Programowanie dynamiczne
    for device in range(1, num_devices + 1):  # Przechodzimy po urządzeniach
        for budget in range(total_budget + 1):  # Przechodzimy po dostępnych kwotach
            for spent in amounts:
                if budget - spent >= 0:
                    prob_device = probabilities[f'U{device}'][spent]
                    new_prob = dp[device - 1, budget - spent] * prob_device
                    if new_prob > dp[device, budget]:
                        dp[device, budget] = new_prob
                        allocation[device, budget] = spent

    # Maksymalne prawdopodobieństwo braku awarii całego obiektu
    max_probability = dp[num_devices, total_budget]

    # Odtwarzanie optymalnego przydziału środków
    budget_left = total_budget
    optimal_allocation = []
    for device in range(num_devices, 0, -1):
        spent = allocation[device, budget_left]
        optimal_allocation.append((f'U{device}', spent))
        budget_left -= spent
    
    # Zwracamy maksymalne prawdopodobieństwo oraz przydział środków na urządzenia
    optimal_allocation.reverse()
    return max_probability, optimal_allocation

# Obliczamy maksymalne prawdopodobieństwo i przydział środków
max_probability, optimal_allocation = max_reliability_with_allocation(total_budget, probabilities)

# Wyświetlamy wyniki
print(f'Maksymalne prawdopodobieństwo braku awarii: {max_probability:.4f}')
print("Optymalny przydział środków:")
for device, spent in optimal_allocation:
    print(f'{device}: {spent} tys. PLN')

Maksymalne prawdopodobieństwo braku awarii: 0.0900
Optymalny przydział środków:
U1: 3 tys. PLN
U2: 2 tys. PLN
U3: 2 tys. PLN
U4: 3 tys. PLN


# Zadanie 3

In [17]:
# -------- Optymalny magazyn – kod z pełnym, czytelnym outputem --------
c, m, z = 20, 5, 35   # koszt jednostki, koszt magazynowania, cena sprzedaży
T = 30                # liczba okresów decyzyjnych (miesięcy)
CAP = 20              # pojemność magazynu
p = 1/5               # P(D_t = d) dla d=0..4

# ----------------------------------- DP ----------------------------------
V  = [[0]*(CAP+1) for _ in range(T+1)]   # wartości V_t(s)
pi = [[0]*(CAP+1) for _ in range(T)]     # optymalne decyzje x_t(s)

# terminalny koszt
for s in range(CAP+1):
    V[T][s] = -(m+z)*s                   # f_T(s) = -(m+z)s

# wstecz od t = 29 … 0
for t in range(T-1, -1, -1):
    for s in range(CAP+1):
        best_val, best_x = -1e12, 0
        for x in range(CAP - s + 1):     # legalne zamówienia
            reward = (z - c)*x + (z*s if t == 0 else -m*s)
            exp = 0.0
            for d in range(5):           # d = 0..4
                s_next = max(0, s + x - d)
                exp += V[t+1][s_next] * p
            val = reward + exp
            if val > best_val:
                best_val, best_x = val, x
        V[t][s], pi[t][s] = best_val, best_x

# ----------------------- podsumowanie wyników ----------------------------
s0 = 0
optimal_payout = V[0][s0]
x0 = pi[0][s0]

# Jak wygląda taktyka? – sprawdzamy do jakiego poziomu „S” polityka dopełnia
base_stock = []
for t in range(T):
    for S in range(CAP+1):
        if all(pi[t][s] == max(0, S - s) for s in range(CAP+1)):
            base_stock.append(S)
            break

# wykrywamy odcinki z tą samą wartością S
segments = []
current_S = base_stock[0]
start_t = 0
for t, S in enumerate(base_stock + [None], 1):
    if S != current_S:
        segments.append((start_t, t-1, current_S))
        current_S, start_t = S, t

print(f"➤ Optymalne pierwsze zamówienie: x0 = {x0} szt.")
print(f"➤ Optymalna oczekiwana wypłata:  V0(s0=0) = {optimal_payout:.2f}\n")

print("➤ Strategia na kolejne miesiące (order‑up‑to):")
for seg in segments:
    a, b, S = seg
    if S == 0:
        rule = "nie kupuj nic"
    else:
        rule = f"zamów do {S} szt. (x_t = max(0, {S} − s_t))"
    if a == b:
        label = f"miesiąc {a}"
    else:
        label = f"miesiące {a}–{b}"
    print(f"  • {label}: {rule}")
print(base_stock)



➤ Optymalne pierwsze zamówienie: x0 = 3 szt.
➤ Optymalna oczekiwana wypłata:  V0(s0=0) = 613.80

➤ Strategia na kolejne miesiące (order‑up‑to):
  • miesiące 0–29: zamów do 3 szt. (x_t = max(0, 3 − s_t))
  • miesiąc 30: zamów do 1 szt. (x_t = max(0, 1 − s_t))
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1]


### Skąd to wynika? 

1. **Programowanie dynamiczne (DP)**  
   W każdym miesiącu $t$ algorytm wybiera zamówienie $x$, które maksymalizuje  

   $$
     Q_t(s,x)=f_t(s,x)+\mathbb{E}\bigl[V_{t+1}(s_{t+1})\bigr].
   $$

2. **Ostatni miesiąc $t=29$**  
   Po zakończeniu horyzontu niesprzedana sztuka kosztuje  

   $$
     -(m+z)=-(5+35)=-40.
   $$

   | dodatkowa sztuka | $\Pr(\text{sprzeda się})$ | zysk $+15$ | kara $-40$ | wartość oczekiwana |
   |:----------------:|:-------------------------:|:----------:|:----------:|:------------------:|
   | 1-sza | $P(D_t\ge1)=0.8$ | $+15$ | $-40$ | $0.8\cdot15\;-\;0.2\cdot40 = \mathbf{+4}$ |
   | 2-ga  | $P(D_t\ge2)=0.6$ | $+15$ | $-40$ | $0.6\cdot15\;-\;0.4\cdot40 = -1$ |
   | 3-cia | $P(D_t\ge3)=0.4$ | $+15$ | $-40$ | $0.4\cdot15\;-\;0.6\cdot40 = -9$ |

   Pierwsza sztuka ma dodatnią wartość oczekiwaną $(+4)$; druga i kolejne już ujemną.  
   $\Longrightarrow$ w ostatnim okresie **uzupełniamy zapas co najwyżej do 1 sztuki**.
